For convenience, let's begin by enabling [automatic reloading of modules](https://ipython.readthedocs.io/en/stable/config/extensions/autoreload.html?highlight=autoreload) when they change.

In [1]:
%load_ext autoreload
%autoreload 2

# Import Qiskit Metal

In [2]:
import qiskit_metal as metal
from qiskit_metal import designs, draw
from qiskit_metal import MetalGUI, Dict, Headings
from qiskit_metal.components.qubits.transmon_pocket import TransmonPocket
from qiskit_metal.components.qubits.transmon_cross import TransmonCross

In [3]:
design = designs.DesignPlanar()
gui = MetalGUI(design)

In [4]:
design.overwrite_enabled = True

In [5]:
Headings.h1('Can view the template_options for QComponents and renderers within design.')

In [6]:
# This populates the template options without making an compnent.
print(TransmonPocket.get_template_options(design))
TransmonPocket.get_template_options(design)

{'pos_x': '0um', 'pos_y': '0um', 'connection_pads': {}, '_default_connection_pads': {'pad_gap': '15um', 'pad_width': '125um', 'pad_height': '30um', 'pad_cpw_shift': '5um', 'pad_cpw_extent': '25um', 'cpw_width': 'cpw_width', 'cpw_gap': 'cpw_gap', 'cpw_extend': '100um', 'pocket_extent': '5um', 'pocket_rise': '65um', 'loc_W': '+1', 'loc_H': '+1'}, 'pad_gap': '30um', 'inductor_width': '20um', 'pad_width': '455um', 'pad_height': '90um', 'pocket_width': '650um', 'pocket_height': '650um', 'orientation': '0', 'ansys_inductance': '10nH', 'ansys_capacitance': 0, 'ansys_resistance': 0, 'ansys_mesh_kw_jj': 7e-06, 'gds_cell_name': 'my_other_junction'}


{'pos_x': '0um',
 'pos_y': '0um',
 'connection_pads': {},
 '_default_connection_pads': {'pad_gap': '15um',
  'pad_width': '125um',
  'pad_height': '30um',
  'pad_cpw_shift': '5um',
  'pad_cpw_extent': '25um',
  'cpw_width': 'cpw_width',
  'cpw_gap': 'cpw_gap',
  'cpw_extend': '100um',
  'pocket_extent': '5um',
  'pocket_rise': '65um',
  'loc_W': '+1',
  'loc_H': '+1'},
 'pad_gap': '30um',
 'inductor_width': '20um',
 'pad_width': '455um',
 'pad_height': '90um',
 'pocket_width': '650um',
 'pocket_height': '650um',
 'orientation': '0',
 'ansys_inductance': '10nH',
 'ansys_capacitance': 0,
 'ansys_resistance': 0,
 'ansys_mesh_kw_jj': 7e-06,
 'gds_cell_name': 'my_other_junction'}

In [7]:
design.delete_all_components() 
gui.rebuild() # refresh

In [8]:
from qiskit_metal.components.qubits.transmon_pocket import TransmonPocket

# Allow running the same cell here multiple times to overwrite changes
design.overwrite_enabled = True

## Custom options for all the transmons
options = dict(
    # Some options we want to modify from the deafults
    # (see below for defaults)
    pad_width = '425 um', 
    pocket_height = '650um',
    # Adding 4 connectors (see below for defaults)
    connection_pads=dict( 
        a = dict(loc_W=+1,loc_H=+1), 
        b = dict(loc_W=-1,loc_H=+1, pad_height='30um'),
        c = dict(loc_W=+1,loc_H=-1, pad_width='200um'),
        d = dict(loc_W=-1,loc_H=-1, pad_height='50um')
    )
)

## Create 4 transmons

q1 = TransmonPocket(design, 'Q1', options = dict(
    pos_x='+2.55mm', pos_y='+0.0mm', gds_cell_name='FakeJunction_02', **options))
q2 = TransmonPocket(design, 'Q2', options = dict(
    pos_x='+0.0mm', pos_y='-0.9mm', orientation = '90', gds_cell_name='FakeJunction_02', **options))
q3 = TransmonPocket(design, 'Q3', options = dict(
    pos_x='-2.55mm', pos_y='+0.0mm', gds_cell_name='FakeJunction_01',**options))
q4 = TransmonPocket(design, 'Q4', options = dict(
    pos_x='+0.0mm', pos_y='+0.9mm', orientation = '90', gds_cell_name='my_other_junction', **options))

## Rebuild the design
gui.rebuild()
gui.autoscale()

In [9]:
Headings.h1('Connecting QPins with coplanar waveguides (CPWs)')

Let's import the basic cpw QComponent from the QLibrary. It is a class called `RouteMeander`.
We can see its default options using `RouteMeander.get_template_options(design)`

In [10]:
from qiskit_metal.components.interconnects.meandered import RouteMeander
RouteMeander.get_template_options(design)

{'pin_inputs': {'start_pin': {'component': '', 'pin': ''},
  'end_pin': {'component': '', 'pin': ''}},
 'fillet': '0',
 'lead': {'start_straight': '0mm',
  'end_straight': '0mm',
  'start_jogged_extension': '',
  'end_jogged_extension': ''},
 'total_length': '7mm',
 'chip': 'main',
 'layer': '1',
 'trace_width': 'cpw_width',
 'meander': {'spacing': '200um', 'asymmetry': '0um'},
 'snap': 'true',
 'prevent_short_edges': 'true'}

We can now modify the options and connect all four qubits. Since this is repetative, you can define a function to wrap up the repetatvie steps. Here we will call this `connect`. This function creates a `RouteMeander` QComponent class.

In [11]:
options = Dict(
    meander=Dict(
        lead_start='0.1mm',
        lead_end='0.1mm',
        asymmetry='0 um')
)


def connect(component_name: str, component1: str, pin1: str, component2: str, pin2: str,
            length: str, asymmetry='0 um', flip=False, fillet='50um'):
    """Connect two pins with a CPW."""
    myoptions = Dict(
        fillet=fillet,
        pin_inputs=Dict(
            start_pin=Dict(
                component=component1,
                pin=pin1),
            end_pin=Dict(
                component=component2,
                pin=pin2)),
        lead=Dict(
            start_straight='0.13mm',
            end_straight='0.13mm'
        ),
        total_length=length)
    myoptions.update(options)
    myoptions.meander.asymmetry = asymmetry
    myoptions.meander.lead_direction_inverted = 'true' if flip else 'false'
    return RouteMeander(design, component_name, myoptions)


asym = 90
cpw1 = connect('cpw1', 'Q1', 'd', 'Q2', 'c', '6.0 mm', f'+{asym}um', fillet='25um')
cpw2 = connect('cpw2', 'Q3', 'c', 'Q2', 'a', '6.1 mm', f'-{asym}um', flip=True, fillet='100um')
cpw3 = connect('cpw3', 'Q3', 'a', 'Q4', 'b', '6.0 mm', f'+{asym}um', fillet='75um')
cpw4 = connect('cpw4', 'Q1', 'b', 'Q4', 'd', '6.1 mm', f'-{asym}um', flip=True)

gui.rebuild()
gui.autoscale()


In [12]:
Headings.h1('Exporting a GDS file.')

In [14]:
#QDesign enables GDS renderer during init.
a_gds = design.renderers.gds
# An alternate way to envoke the command without using a_gds.
# design.renderers.gds.export_to_gds()

#Show the options for GDS
a_gds.options

{'short_segments_to_not_fillet': 'True',
 'check_short_segments_by_scaling_fillet': '2.0',
 'gds_unit': 0.001,
 'ground_plane': 'True',
 'corners': 'circular bend',
 'tolerance': '0.00001',
 'precision': '0.000000001',
 'width_LineString': '10um',
 'path_filename': '../resources/Fake_Junctions.GDS',
 'junction_pad_overlap': '5um',
 'max_points': '8191',
 'bounding_box_scale_x': '1.2',
 'bounding_box_scale_y': '1.2'}

To made juction table work correctly, GDS Renderer needs corect path to gds file which has cells.  
Each cell is a junction to be placed in a Transmon.  A sample gds file is provided in directory qiskit_metal/guide/resources.
There are three cells with names "Fake_Junction_01", "Fake_Junction_01", and "my_other_junction".
The default name used by GDS Render is "my_other_junction".  If you want to customize the selection,
you can pass it during qcomponent being added to QDesign through the options. 

In [15]:
a_gds.options['path_filename'] = '../../resources/Fake_Junctions.GDS'

Do you want GDS Renderer to fix any short-segments in your QDesign when using fillet?'



In [16]:
#If you have a fillet_value and there are LineSegments that are shorter than 2*fillet_value, 
#When true, the short segments will not be fillet'd. 
a_gds.options['short_segments_to_not_fillet'] = 'False'

#If you want to 
a_gds.options['short_segments_to_not_fillet'] = 'True'
scale_fillet = 2.0
a_gds.options['check_short_segments_by_scaling_fillet'] = scale_fillet

In [17]:
# Export GDS file for all components in design.
#def export_to_gds(self, file_name: str, highlight_qcomponents: list = []) -> int:


# Please change the path where you want to write a GDS file.
#Example below.  
#a_gds.export_to_gds("../../../gds-files/GDS QRenderer Notebook.gds")
a_gds.export_to_gds("GDS QRenderer Notebook.gds")




1

In [ ]:
# Export a GDS file which contains only few components.
a_gds.export_to_gds("../../../gds-files/four_qcomponents.gds",
                           highlight_qcomponents=['cpw1', 'cpw4', 'Q1', 'Q3'])



In [ ]:
# Export a GDS file using explicit path and cpw1.name vs typing string.
a_gds.export_to_gds("../../../gds-files/four_same_qcomponents.gds",
                           highlight_qcomponents=[cpw1.name, 'cpw4', q1.name, 'Q3'])

We can see all the pins

In [ ]:
gui.highlight_components(['Q1','Q2','Q3','Q4','cpw1','cpw2','cpw3','cpw4'])
gui.screenshot()

In [ ]:
Headings.h1('Variables in options')

## Variables 

The design can have variables, which can be used in the component options.

In [ ]:
design.variables.cpw_width = '10um'
design.variables.cpw_gap = '6um'

For example, we can add qubit pads using variables.  

In [ ]:
# Set variables in the design
design.variables.pad_width = '550 um'

# Assign variables to component options
q1.options.pad_width = 'pad_width'
q2.options.pad_width = 'pad_width'
q3.options.pad_width = 'pad_width'

# Rebuild all compoinent and refresh the gui 
gui.rebuild()
gui.autoscale()
gui.screenshot()

In [ ]:
Headings.h1('More QComponents')

In [ ]:
from qiskit_metal.components.basic.n_square_spiral import NSquareSpiral
# print(NSquareSpiral.get_template_options(design))
ops = {
    'n': '10',
    'width': '10um',
    'radius': '100um',
    'gap': '22um',
    'pos_x': '0.65mm',
    'pos_y': '2.2mm',
    'rotation': '0',
    'subtract': 'False'}
NSquareSpiral(design, 'spiral', ops)
NSquareSpiral(design, 'spiral_cut', {**ops, **dict(subtract=True, width='22um', gap='10um')})
gui.rebuild()

In [ ]:
from qiskit_metal.components.interconnects.straight_path import RouteStraight
# RouteStraight.get_template_options(design)
myoptions = Dict(
        pin_inputs=Dict(
            start_pin=Dict(
                component='Q4',
                pin='c'),
            end_pin=Dict(
                component='spiral',
                pin='spiralPin'))
        )


RouteStraight(design, 'cpw_s1', myoptions);
gui.rebuild()

In [ ]:
qcomponents = ['spiral', 'cpw_s1']
gui.highlight_components(qcomponents)
# gui.zoom_on_components(qcomponents)
gui.screenshot()

####  NGon

In [ ]:
from qiskit_metal.components.basic.n_gon import NGon
# display(NGon.get_template_options(design))
ops = {
    'n': '5',
    'radius': '250um',
    'pos_x': '-0.85mm',
    'pos_y': '2.0mm',
    'rotation': '15',
    'subtract': 'False',
    'helper': 'False',
    'chip': 'main',
    'layer': '1',
    'width': '10um'}
NGon(design, 'ngon', ops)
NGon(design, 'ngon_negative', {**ops, **dict(subtract=True, radius='350um')})
gui.rebuild()

In [ ]:
gui.zoom_on_components(['ngon_negative'])
gui.screenshot()

In [ ]:
from qiskit_metal.components.basic.circle_raster import CircleRaster
display(CircleRaster.get_template_options(design))

ops = { 'radius': '300um',
        'pos_x': '-1.5mm',
        'pos_y': '2mm',
        'resolution': '16',
        'cap_style': 'round',
        'subtract': 'False',
        'helper': 'False',
        'chip': 'main',
        'layer': '1',
        'width': '10um'}
CircleRaster(design, 'CircleRaster', ops)
gui.rebuild()

In [ ]:
gui.zoom_on_components(['CircleRaster'])
gui.screenshot()

In [ ]:
from qiskit_metal.components.basic.rectangle_hollow import RectangleHollow
display(RectangleHollow.get_template_options(design))

ops = { 'width': '500um',
        'height': '300um',
        'pos_x': '-2.3mm',
        'pos_y': '2mm',
        'rotation': '0',
        'subtract': 'False',
        'helper': 'False',
        'chip': 'main',
        'layer': '1',
        'inner': {  'width': '250um',
                    'height': '100um',
                    'offset_x': '40um',
                    'offset_y': '-20um',
                    'rotation': '15'}}
RectangleHollow(design, 'RectangleHollow', ops)
gui.rebuild()

In [ ]:
gui.zoom_on_components(['RectangleHollow'])
gui.screenshot()

In [ ]:
gui.autoscale()
gui.screenshot()

In [ ]:
a_gds.export_to_gds("../../../gds-files/StartHere_sample_qcomponents.gds")



In [ ]:
Headings.h1('The geometry of QComponent: QGeometry')

### Geometric boundary of a qcomponent?
Return the boundry box of the geometry, for example: `q1.qgeometry_bounds()`.  
The function returns a tuple containing (minx, miny, maxx, maxy) bound values
for the bounds of the component as a whole.

In [ ]:
for name, qcomponent in design.components.items():
    print(f"{name:10s} : {qcomponent.qgeometry_bounds()}")

### What is QGeometry? 

We can get all the QGeometry of a QComponent. There are several kinds, such as `path` and `poly`. Let us look at all the polygons used to create qubit `q1`

In [ ]:
q1.qgeometry_table('poly')

Paths are lines. These can have a width.

In [ ]:
q1.qgeometry_table('path')

In [ ]:
Headings.h1('Qiskit Metal Version')

In [ ]:
metal.about();